In [ ]:
import comet_ml
from comet_ml.query import Parameter
import pandas as pd
import numpy as np

comet_api = comet_ml.api.API()

def get_exps(strat):
    query = (Parameter('SERVER_OPT') == 'Adam') & (Parameter('CLIENT_OPT') == 'Adam') & (Parameter('CLIENT_OPT_STRATEGY') == strat)
    return comet_api.query('federated-learning', 'both-side-opt', query)

def get_df(exps):
    def get_list(t, c):
        res = []
        for exp in exps:
            response = c(exp)
            if isinstance(response, list):
                res.append(None)
            else:
                res.append(t(response['valueMax']))
        return res
    def get_params(t, n):
        return get_list(t, lambda exp: exp.get_parameters_summary(n))
    def get_metrics(t, n):
        return get_list(t, lambda exp: exp.get_metrics_summary(n))

    ids = [exp.id for exp in exps]

    c_lr = get_params(float, 'CLIENT_LEARNING_RATE')
    s_lr = get_params(float, 'SERVER_LEARNING_RATE')

    last100_avg_acc = get_metrics(float, 'last100_avg_acc')
    teas = get_metrics(float, 'test_acc')
    return pd.DataFrame({'id': ids, 
                         'test_acc': teas, 
                         'last100_avg_acc': last100_avg_acc, 
                         'c_lr': c_lr, 
                         's_lr': s_lr,
                            })
exps = get_exps('nothing')
df = get_df(exps)